In [ ]:
!pip install inference supervision==0.26.0 pillow --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.7/206.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: supervision
    Found existing installation: supervision 0.27.0.post1
    Uninstalling supervision-0.27.0.post1:
      Successfully uninstalled supervision-0.27.0.post1


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d92ab8e088728ad9a3d653f0ad160d13bf000f324063cdd869ac13e7372c62aa
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install structlog backoff redis pybase64 tldextract onnxruntime
import os
import supervision as sv
from inference import get_model
from PIL import Image

# -------- CONFIG --------
INPUT_FOLDER = "/content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/train/images"
OUTPUT_FOLDER_BASE = "/content/drive/MyDrive/output_dog_rfdetr_base"
OUTPUT_FOLDER_LARGE = "/content/drive/MyDrive/output_rfdetr_large"
CONFIDENCE = 0.5
# ------------------------

os.makedirs(OUTPUT_FOLDER_BASE, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_LARGE, exist_ok=True)

print("Loading models...")
model_base = get_model("rfdetr-base")
model_large = get_model("rfdetr-large")
print("Models loaded successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 59.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inference 1.0.1 requires aiortc>=1.9.0, which is not installed.
inference 1.0.1 requires anthropic~=0.49.0, which is not installed.
inference 1.0.1 requires APScheduler<4.0.0,>=3.10.1, which is not installed.
inference 1.0.1 requires asyncua~=1.1.5, which is not installed.
inference 1.0.1 requires av==14.2.0, which is not installed.
inference 1.0.1 requires boto3<=1.41.5,>=1.40.0, which is not installed.
inference 1.0.1 requires botocore<=1.41.5,>=1.40.0, which is not installed.
inference 1.0.1 requires dataclasses-json~=0.6.0, which is not installed.
inference 1.0.1 requires docker<8.0.0,>=7.0.0, which is not installed.
inference 1.0.1 requires inference-models~=0.19.1, which is not installed.
inference 1.0.1 requires nvidia-ml-py<13.0

ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM2 model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM2_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM3 model. Install SAM3 dependencies and set CORE_MODEL_SAM3_ENABLED to True.
ModelDependencyMissing: Your `inference` configuration does not support CLIP model. Use pip install 'inference[clip]' to install missing requirements.To suppress this warning, set CORE_MODEL_CLIP_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install missing requirements.To suppress this warning, set CORE_MODEL_GAZE_ENABL

Loading models...


Models loaded successfully.


In [ ]:
import os
import supervision as sv
from inference import get_model
from PIL import Image

# -------- CONFIG --------
INPUT_FOLDER = "/content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/train/images"
OUTPUT_FOLDER_BASE = "/content/drive/MyDrive/output_rfdetr_base"
OUTPUT_FOLDER_LARGE = "/content/drive/MyDrive/output_rfdetr_large"
CONFIDENCE = 0.5
# ------------------------

os.makedirs(OUTPUT_FOLDER_BASE, exist_ok=True)
os.makedirs(OUTPUT_FOLDER_LARGE, exist_ok=True)

print("Loading models...")
model_base = get_model("rfdetr-base")
model_large = get_model("rfdetr-large")
print("Models loaded successfully.")

valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

for filename in os.listdir(INPUT_FOLDER):

    if not filename.lower().endswith(valid_ext):
        continue

    image_path = os.path.join(INPUT_FOLDER, filename)

    try:
        image = Image.open(image_path).convert("RGB")

        # ---------- BASE ----------
        pred_base = model_base.infer(image, confidence=CONFIDENCE)[0]
        det_base = sv.Detections.from_inference(pred_base)
        labels_base = [p.class_name for p in pred_base.predictions]

        annotated_base = image.copy()
        annotated_base = sv.BoxAnnotator().annotate(annotated_base, det_base)
        annotated_base = sv.LabelAnnotator().annotate(
            annotated_base, det_base, labels_base
        )

        annotated_base.save(os.path.join(OUTPUT_FOLDER_BASE, filename))

        # ---------- LARGE ----------
        pred_large = model_large.infer(image, confidence=CONFIDENCE)[0]
        det_large = sv.Detections.from_inference(pred_large)
        labels_large = [p.class_name for p in pred_large.predictions]

        annotated_large = image.copy()
        annotated_large = sv.BoxAnnotator().annotate(annotated_large, det_large)
        annotated_large = sv.LabelAnnotator().annotate(
            annotated_large, det_large, labels_large
        )

        annotated_large.save(os.path.join(OUTPUT_FOLDER_LARGE, filename))

        print(f"✅ Processed: {filename}")

    except Exception as e:
        print(f"❌ Error with {filename}: {e}")

print("🎯 All images processed.")

Loading models...


Models loaded successfully.
✅ Processed: images-11-_jpg.rf.cb278c0e414f9e954ed24115ae1d72f0.jpg
✅ Processed: download-3-_jpg.rf.55732aebaf4cf9b7eb6c69eb32daedf2.jpg
✅ Processed: download-20-_jpg.rf.67d720fa6831816e51bab89991bff6dd.jpg
✅ Processed: download-4-_jpg.rf.c014fb99948dcc00575b2eb0730f00f6.jpg
✅ Processed: download-3-_jpg.rf.23c7c9674b88946e074678fdfcf1f01d.jpg
✅ Processed: download-4-_jpg.rf.eab782b5ec4692d16769855a983b2ceb.jpg
✅ Processed: download-21-_jpg.rf.24ebc20cec9c3c3a2b3e68f9f6ac178d.jpg
✅ Processed: images-11-_jpg.rf.84d243c8143da5d9c8f742241324d1fb.jpg
✅ Processed: download-5-_jpg.rf.41ad32100d0671c16d3d73c74ac7cb70.jpg
✅ Processed: download-4-_jpg.rf.c3da2251ad849e620becbd8d34c0a76b.jpg
✅ Processed: images-11-_jpg.rf.97d98fb1346affe7f25576b9d5c8ebc3.jpg
✅ Processed: download-20-_jpg.rf.891d39b33485958816e393a54b05a0f3.jpg
✅ Processed: download-21-_jpg.rf.c8b8e42cf14dc4cfff236f0901787371.jpg
✅ Processed: download-6-_jpg.rf.1fc5b25b954f58611622d0ad4edeb68c.jpg
✅ Pro

KeyboardInterrupt: 

In [1]:
!pip install ultralytics
!pip install sam2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.4 MB/s eta 0:00:00
  Created wheel for sam2: filename=sam2-1.1.0-cp312-cp312-linux_x86_64.whl size=505268 sha256=b7d45565d887638c980a9faac0732e29c20a2fbb18982736233098ad36ac7f00
  Stored in directory: /root/.cache/pip/wheels/22/cb/54/0351e695a7a2e8eed8d5987cd02d8e5227eec87b0bbcc00094
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=1de3f153b2c545e2abc307f0a30308f42374e820d82347e3d610b297bb436da5
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb163435781

In [2]:
# Install latest versions
!pip install -U ultralytics supervision

import os
import requests
import shutil # Import shutil for moving files

# Direct links for SAM 2.1 Hiera Large (Late 2024/2025 Release)
files = {
    "sam2.1_hiera_large.pt": "https://huggingface.co/facebook/sam2.1-hiera-large/resolve/main/sam2.1_hiera_large.pt",
    "sam2.1_hiera_l.yaml": "https://raw.githubusercontent.com/facebookresearch/segment-anything-2/main/sam2_configs/sam2.1/sam2.1_hiera_l.yaml"
}

for name, url in files.items():
    if not os.path.exists(name):
        print(f"Downloading {name}...")
        r = requests.get(url)
        with open(name, 'wb') as f:
            f.write(r.content)

# Define the target directory within the installed sam2 package for the config file
sam2_config_target_dir = "/usr/local/lib/python3.12/dist-packages/sam2/sam2_configs/sam2.1/"

# Create the target directory if it doesn't exist
os.makedirs(sam2_config_target_dir, exist_ok=True)

# Move the downloaded config file to the target directory
source_config_path = "sam2.1_hiera_l.yaml"
dest_config_path = os.path.join(sam2_config_target_dir, "sam2.1_hiera_l.yaml")

if os.path.exists(source_config_path) and not os.path.exists(dest_config_path):
    print(f"Moving {source_config_path} to {dest_config_path}...")
    shutil.move(source_config_path, dest_config_path)
    print("✅ Config file moved successfully.")
elif os.path.exists(dest_config_path):
    print(f"Config file already exists at {dest_config_path}.")
else:
    print(f"❌ Source config file {source_config_path} not found.")

print("✅ Environment Ready.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.4/217.4 kB 12.0 MB/s eta 0:00:00
Moving sam2.1_hiera_l.yaml to /usr/local/lib/python3.12/dist-packages/sam2/sam2_configs/sam2.1/sam2.1_hiera_l.yaml...
✅ Config file moved successfully.
✅ Environment Ready.


In [ ]:
import os
import cv2
import torch
import numpy as np
import supervision as sv
from ultralytics import YOLO
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

from ultralytics import YOLO, SAM

# Load models (Ultralytics handles the download automatically)
yolo_model = YOLO("yolo11n.pt")
sam_model = SAM("sam2_l.pt")  # This replaces build_sam2 and the predictor

# -------- CONFIG --------
INPUT_FOLDER = "/content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/train/test"
OUTPUT_FOLDER_SAM = "/content/drive/MyDrive/output_yolo11_sam2"
CONFIDENCE = 0.5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# SAM 2 Checkpoint & Config (Ensure these files are in your directory)
sam2_checkpoint = "sam2_hiera_large.pt"
model_cfg = "sam2_hiera_l.yaml"
# ------------------------

os.makedirs(OUTPUT_FOLDER_SAM, exist_ok=True)

# 1. Load YOLO11 for Detection
print("Loading YOLO11...")
yolo_model = YOLO("yolo11n.pt") # or your custom trained dog model

# 2. Load SAM 2 for Segmentation
print("Loading SAM 2...")
sam2_model = build_sam2(model_cfg, sam2_checkpoint, device=DEVICE)
sam_predictor = SAM2ImagePredictor(sam2_model)

valid_ext = (".jpg", ".jpeg", ".png", ".bmp", ".webp")

for filename in os.listdir(INPUT_FOLDER):
    if not filename.lower().endswith(valid_ext):
        continue

    image_path = os.path.join(INPUT_FOLDER, filename)

    try:
        # Load image for both models
        raw_image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)

        # --- STAGE 1: YOLO11 Detection ---
        results = yolo_model.predict(image_rgb, conf=CONFIDENCE)[0]
        detections = sv.Detections.from_ultralytics(results)

        # Filter for 'dog' class only (COCO class 16 is dog)
        detections = detections[detections.class_id == 16]

        if len(detections) > 0:
            # --- STAGE 2: SAM 2 Segmentation ---
            sam_predictor.set_image(image_rgb)

            # Use YOLO bounding boxes as prompts for SAM
            boxes = detections.xyxy
            masks, scores, _ = sam_predictor.predict(
                box=boxes,
                multimask_output=False
            )

            # Update supervision detections with SAM masks
            detections.mask = np.array(masks).squeeze()

            # --- STAGE 3: 3D-Ready Visualization ---
            mask_annotator = sv.MaskAnnotator()
            box_annotator = sv.BoxAnnotator()
            label_annotator = sv.LabelAnnotator()

            annotated_image = mask_annotator.annotate(scene=raw_image.copy(), detections=detections)
            annotated_image = box_annotator.annotate(scene=annotated_image, detections=detections)
            annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

            cv2.imwrite(os.path.join(OUTPUT_FOLDER_SAM, filename), annotated_image)
            print(f"✅ SAM Segmentation Complete: {filename}")
        else:
            print(f"ℹ️ No dogs detected in {filename}")

    except Exception as e:
        print(f"❌ Error with {filename}: {e}")

print("🎯 Pipeline processing finished.")

Loading YOLO11...
Loading SAM 2...


FileNotFoundError: [Errno 2] No such file or directory: 'sam2_hiera_large.pt'

In [ ]:
import os
import cv2
import supervision as sv
from ultralytics import YOLO, SAM
import numpy as np

# -------- CONFIG --------
INPUT_FOLDER = "/content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/test/images"
OUTPUT_FOLDER = "/content/drive/MyDrive/output_yolo11_sam2_final"
CONFIDENCE = 0.5
DOG_CLASS_ID = 16
# ------------------------

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Loading models
yolo_model = YOLO("yolo11n.pt")
sam_model = SAM("sam2.1_l.pt")

def analyze_posture(mask):
    """Calculates density and shape metrics from a binary mask."""
    # Area of the dog in pixels
    area = np.sum(mask)

    # Calculate bounding box dimensions from mask
    y, x = np.where(mask)
    if len(x) == 0: return 0, 0, "Unknown"

    height = np.max(y) - np.min(y)
    width = np.max(x) - np.min(x)
    aspect_ratio = height / width if width > 0 else 0

    # Simple logic for posture
    if aspect_ratio < 0.6:
        posture = "Lying Down"
    elif aspect_ratio > 1.5:
        posture = "Standing/Walking"
    else:
        posture = "Alert/Sitting"

    return area, aspect_ratio, posture

for filename in os.listdir(INPUT_FOLDER):
    if not filename.lower().endswith((".jpg", ".png", ".jpeg")): continue

    image_path = os.path.join(INPUT_FOLDER, filename)
    image_bgr = cv2.imread(image_path)
    if image_bgr is None: continue

    # Step 1: YOLO Detection
    results_yolo = yolo_model.predict(source=image_bgr, conf=CONFIDENCE, classes=[DOG_CLASS_ID], verbose=False)[0]

    if len(results_yolo.boxes) > 0:
        # Step 2: SAM Segmentation
        bboxes = results_yolo.boxes.xyxy.tolist()
        results_sam = sam_model.predict(source=image_bgr, bboxes=bboxes, verbose=False)[0]

        detections = sv.Detections.from_ultralytics(results_sam)

        # Analyze each detected dog
        custom_labels = []
        for i, mask in enumerate(detections.mask):
            area, ratio, posture = analyze_posture(mask)
            # Dog Density = (Dog Area / Total Image Area) * 100
            total_pixels = image_bgr.shape[0] * image_bgr.shape[1]
            density = (area / total_pixels) * 100

            label = f"{posture} | Density: {density:.2f}%"
            custom_labels.append(label)

        # Step 3: Annotation
        mask_annotator = sv.MaskAnnotator()
        label_annotator = sv.LabelAnnotator(text_scale=0.5, text_thickness=1)

        annotated_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
        annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=custom_labels)

        cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), annotated_image)
        print(f"✅ Analyzed: {filename} - Found {len(custom_labels)} dogs.")

print("🎯 Analysis Pipeline Complete.")

✅ Analyzed: 44-552x368_png.rf.ca58db044bdcdb4d2ac80f394ddc9d14.jpg - Found 1 dogs.
✅ Analyzed: dogs18_jpg.rf.0834cf944b9c16bc637bf0a6af72942a.jpg - Found 2 dogs.
✅ Analyzed: dogs23_jpg.rf.a2ee25cb3af2773cd18db39574275cde.jpg - Found 1 dogs.
✅ Analyzed: dogs16_jpg.rf.ab97c10755b4025767cf208c3c08159d.jpg - Found 4 dogs.
✅ Analyzed: 1614200312166314289_png.rf.69a674f0b2c95a52f3aa8b2ece7e38fd.jpg - Found 1 dogs.
✅ Analyzed: caucel11_jpg.rf.dfd370338dcb2c0d9076c234780a8ed7.jpg - Found 1 dogs.
✅ Analyzed: dogs_207_jpg.rf.0b0e2ba421f315ff17165854b78fa768.jpg - Found 2 dogs.
✅ Analyzed: dogs_165_jpg.rf.4912aea1717ffa2b42161e0762df0d1d.jpg - Found 2 dogs.
✅ Analyzed: dogs_215_jpg.rf.34f5fae995c996ac8757db689ae6801d.jpg - Found 1 dogs.
✅ Analyzed: dogs_107_jpg.rf.76e606a8691f8baa94c6b9dbb923ff92.jpg - Found 1 dogs.
✅ Analyzed: dogs_207_jpg.rf.c1abf09f648f5d2abb7148b3dee813ea.jpg - Found 4 dogs.
✅ Analyzed: dogs_274_jpg.rf.2b7832ddcc14bd97cc0ede2676434fc1.jpg - Found 2 dogs.
✅ Analyzed: d24_jpg.r

In [8]:
import os
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO, SAM

# -------- CONFIG --------
INPUT_PATH = "/content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/test/images" # Folder with images OR a .mp4 file
OUTPUT_FOLDER = "/content/drive/MyDrive/output_adaptive"
CONFIDENCE = 0.5
DOG_CLASS_ID = 16
# ------------------------

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Loading the 2026 SOTA models
yolo_model = YOLO("yolo11n.pt")
sam_model = SAM("sam2.1_l.pt")

def is_mischievous(mask):
    """Heuristic logic to flag mischievous postures."""
    y, x = np.where(mask)
    if len(x) == 0: return False
    height, width = np.max(y) - np.min(y), np.max(x) - np.min(x)
    aspect_ratio = height / width if width > 0 else 0
    # Flag if dog is jumping (>1.8) or lunging/running (<0.4)
    return aspect_ratio > 1.8 or aspect_ratio < 0.4

def process_frame(frame):
    """Core logic to process a single frame for dogs and mischief."""
    results_yolo = yolo_model.predict(source=frame, conf=CONFIDENCE, classes=[DOG_CLASS_ID], verbose=False)[0]

    if len(results_yolo.boxes) > 0:
        all_boxes = results_yolo.boxes.xyxy.tolist()
        results_sam = sam_model.predict(source=frame, bboxes=all_boxes, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results_sam)

        # Filter for mischief
        mischief_mask = np.array([is_mischievous(m) for m in detections.mask])

        # Annotators
        box_annotator = sv.BoxAnnotator(color=sv.Color.WHITE)
        mask_annotator = sv.MaskAnnotator(color=sv.Color.RED)
        label_annotator = sv.LabelAnnotator()

        # 1. Annotate ALL boxes
        annotated_frame = box_annotator.annotate(scene=frame.copy(), detections=detections)

        # 2. Annotate ONLY mischief with Red SAM Masks
        if any(mischief_mask):
            m_detections = detections[mischief_mask]
            annotated_frame = mask_annotator.annotate(scene=annotated_frame, detections=m_detections)
            labels = ["MISCHIEF ALERT" for _ in range(len(m_detections))]
            annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=m_detections, labels=labels)

        return annotated_frame
    return frame

# --- ADAPTIVE HANDLER ---
if os.path.isdir(INPUT_PATH):
    # PROCESS IMAGE FOLDER
    print(f"📁 Processing folder: {INPUT_PATH}")
    for filename in os.listdir(INPUT_PATH):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            img = cv2.imread(os.path.join(INPUT_PATH, filename))
            processed = process_frame(img)
            cv2.imwrite(os.path.join(OUTPUT_FOLDER, filename), processed)
            print(f"✅ Processed Image: {filename}")

elif INPUT_PATH.lower().endswith((".mp4", ".avi", ".mov")):
    # PROCESS VIDEO FILE
    print(f"🎥 Processing video: {INPUT_PATH}")
    cap = cv2.VideoCapture(INPUT_PATH)
    width  = int(cap.get(cv2.get(cv2.CAP_PROP_FRAME_WIDTH)))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(os.path.join(OUTPUT_FOLDER, "processed_output.mp4"),
                         cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()
    print("✅ Video processing complete.")

📁 Processing folder: /content/drive/MyDrive/Stray_Detection_Project_Submission_FIle/test/images
✅ Processed Image: dogs48_jpg.rf.32e73bd3a639490f59d23eb65375c3c6.jpg
✅ Processed Image: 44-552x368_png.rf.ca58db044bdcdb4d2ac80f394ddc9d14.jpg
✅ Processed Image: dogs18_jpg.rf.0834cf944b9c16bc637bf0a6af72942a.jpg
✅ Processed Image: dogs46_jpg.rf.fa72e287ef0874bc05cf5d7f37b21320.jpg
✅ Processed Image: dogs23_jpg.rf.a2ee25cb3af2773cd18db39574275cde.jpg
✅ Processed Image: caucel8_jpg.rf.ce7fb03044d54e067d163cacc30792f0.jpg
✅ Processed Image: dogs54_jpg.rf.bbef65ffa751428e6828bc81333eba14.jpg
✅ Processed Image: dogs16_jpg.rf.ab97c10755b4025767cf208c3c08159d.jpg
✅ Processed Image: 1614200312166314289_png.rf.69a674f0b2c95a52f3aa8b2ece7e38fd.jpg
✅ Processed Image: caucel11_jpg.rf.dfd370338dcb2c0d9076c234780a8ed7.jpg
✅ Processed Image: dogs_207_jpg.rf.0b0e2ba421f315ff17165854b78fa768.jpg
✅ Processed Image: dogs_237_jpg.rf.d41950839c9baa6dfb3aec6ff4f2acd8.jpg
✅ Processed Image: dogs_165_jpg.rf.4912a

In [3]:
import os
import cv2
import kagglehub
import torch
import numpy as np
import supervision as sv
from ultralytics import YOLO, SAM
from tqdm import tqdm # For a professional progress bar

# 1. DOWNLOAD DATASET
print("Downloading Kaggle dataset...")
dataset_path = kagglehub.dataset_download("umuttuygurr/videosdog")

# -------- CONFIG --------
OUTPUT_FOLDER = "/content/drive/MyDrive/output_stray_dog_behavior"
CONFIDENCE = 0.5
DOG_CLASS_ID = 16
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
print(f"Using Device: {DEVICE}")
# ------------------------

# 2. LOAD MODELS ON GPU
# Moving models to GPU significantly speeds up the 'predict' step
yolo_model = YOLO("yolo11n.pt").to(DEVICE)
sam_model = SAM("sam2.1_l.pt").to(DEVICE)

def is_mischievous(mask):
    """Heuristic logic to flag mischievous postures."""
    y, x = np.where(mask)
    if len(x) == 0: return False
    height, width = np.max(y) - np.min(y), np.max(x) - np.min(x)
    aspect_ratio = height / width if width > 0 else 0
    return aspect_ratio > 1.8 or aspect_ratio < 0.4

def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # Progress bar for the video
    for _ in tqdm(range(total_frames), desc=f"Processing {os.path.basename(video_path)}", leave=False):
        ret, frame = cap.read()
        if not ret: break

        # YOLO Detection
        results_yolo = yolo_model.predict(source=frame, conf=CONFIDENCE, classes=[DOG_CLASS_ID], verbose=False, device=DEVICE)[0]

        if len(results_yolo.boxes) > 0:
            bboxes = results_yolo.boxes.xyxy.tolist()
            # SAM Segmentation (This is the heavy part that GPU fixes)
            results_sam = sam_model.predict(source=frame, bboxes=bboxes, verbose=False, device=DEVICE)[0]
            detections = sv.Detections.from_ultralytics(results_sam)

            mischief_mask = np.array([is_mischievous(m) for m in detections.mask])

            box_annotator = sv.BoxAnnotator(color=sv.Color.WHITE)
            mask_annotator = sv.MaskAnnotator(color=sv.Color.RED)

            frame = box_annotator.annotate(scene=frame, detections=detections)
            if any(mischief_mask):
                m_detections = detections[mischief_mask]
                frame = mask_annotator.annotate(scene=frame, detections=m_detections)

        out.write(frame)

    cap.release()
    out.release()

# 3. RUN PIPELINE
print("🎬 Starting GPU-accelerated behavioral analysis...")
for root, _, files in os.walk(dataset_path):
    for filename in files:
        if filename.lower().endswith((".mp4", ".avi")):
            input_file = os.path.join(root, filename)
            output_file = os.path.join(OUTPUT_FOLDER, f"gpu_analyzed_{filename}")
            process_video(input_file, output_file)

print("🎯 Pipeline finished.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 1.32G/1.32G [00:11<00:00, 119MB/s]

Extracting files...


Using Device: cuda
🎬 Starting GPU-accelerated behavioral analysis...


KeyboardInterrupt: 

In [4]:
import os
import cv2
import torch
import kagglehub
import numpy as np
import pandas as pd
import supervision as sv
from ultralytics import YOLO, SAM
from tqdm import tqdm

# 1. DOWNLOAD DATASET
print("Downloading Kaggle dataset...")
dataset_path = kagglehub.dataset_download("umuttuygurr/videosdog")

# -------- CONFIG --------
OUTPUT_FOLDER = "/content/drive/MyDrive/Stray_Behavior_Project"
CONFIDENCE = 0.4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FRAME_STRIDE = 2  # Process SAM every 2nd frame to double the speed
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
# ------------------------

# 2. LOAD MODELS (Optimized for GPU)
print(f"Loading Models on {DEVICE}...")
yolo_model = YOLO("yolo11n.pt").to(DEVICE)
sam_model = SAM("sam2.1_l.pt").to(DEVICE)

# Data storage for CSV
csv_logs = []

def get_behavior(mask):
    """Analyze morphology of the SAM mask to determine posture."""
    y, x = np.where(mask)
    if len(x) == 0: return "Unknown"
    height = np.max(y) - np.min(y)
    width = np.max(x) - np.min(x)
    ratio = height / width if width > 0 else 0

    if ratio > 1.7: return "Jumping/Standing"
    if ratio < 0.5: return "Lying/Lunging"
    return "Normal/Alert"

def process_video(video_path, output_path, filename):
    cap = cv2.VideoCapture(video_path)
    w, h = int(cap.get(3)), int(cap.get(4))
    fps, total_frames = int(cap.get(5)), int(cap.get(7))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

    # Professional Annotators
    # MaskAnnotator creates the colored SAM fill
    mask_annotator = sv.MaskAnnotator(opacity=0.4, color_lookup=sv.ColorLookup.INDEX)
    box_annotator = sv.BoxAnnotator(color_lookup=sv.ColorLookup.INDEX)
    label_annotator = sv.LabelAnnotator(text_scale=0.6, text_thickness=2)

    frame_count = 0
    detections = None
    labels = []

    for _ in tqdm(range(total_frames), desc=f"Processing {filename}", leave=False):
        ret, frame = cap.read()
        if not ret: break

        # Step 1: YOLO Detection (Every frame for smoothness)
        if frame_count % FRAME_STRIDE == 0:
            yolo_results = yolo_model.predict(frame, conf=CONFIDENCE, classes=[16], device=DEVICE, verbose=False, half=True)[0]

            if len(yolo_results.boxes) > 0:
                # Step 2: SAM Segmentation (The "Magic" part)
                bboxes = yolo_results.boxes.xyxy.tolist()
                sam_results = sam_model.predict(frame, bboxes=bboxes, device=DEVICE, verbose=False)[0]

                # Create Supervision Detections including masks
                detections = sv.Detections.from_ultralytics(sam_results)

                # Step 3: Behavioral Analysis
                labels = []
                for i in range(len(detections.mask)):
                    behavior = get_behavior(detections.mask[i])
                    labels.append(f"Dog: {behavior}")
                    # Log for CSV
                    csv_logs.append({
                        "Video": filename,
                        "Frame": frame_count,
                        "Behavior": behavior,
                        "Confidence": detections.confidence[i]
                    })

        # Step 4: Layered Visualization
        if detections is not None:
            # Mask first (bottom layer), then Box, then Label (top layer)
            frame = mask_annotator.annotate(scene=frame, detections=detections)
            frame = box_annotator.annotate(scene=frame, detections=detections)
            frame = label_annotator.annotate(scene=frame, detections=detections, labels=labels)

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

# 3. RUN BATCH PROCESSING
print("🎬 Starting Pipeline...")
for root, _, files in os.walk(dataset_path):
    for f in files:
        if f.lower().endswith((".mp4", ".avi", ".mov")):
            input_vid = os.path.join(root, f)
            output_vid = os.path.join(OUTPUT_FOLDER, f"SAM_Analyzed_{f}")
            process_video(input_vid, output_vid, f)

# 4. EXPORT CSV REPORT
df = pd.DataFrame(csv_logs)
if not df.empty:
    csv_out = os.path.join(OUTPUT_FOLDER, "stray_dog_behavior_report.csv")
    df.to_csv(csv_out, index=False)
    print(f"✅ Analysis Report Saved: {csv_out}")

print("🎯 Mission Accomplished.")

Using Colab cache for faster access to the 'videosdog' dataset.
Loading Models on cuda...
🎬 Starting Pipeline...


✅ Analysis Report Saved: /content/drive/MyDrive/Stray_Behavior_Project/stray_dog_behavior_report.csv
🎯 Mission Accomplished.
